In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Run data_processing and batching_data
%run ~/violin-renderer/src/data/batching_data.ipynb
%run ~/violin-renderer/src/models/mlp/data_processing.ipynb

# Initialize GPU to move model/tensors onto
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Trains the model inputted into the function.

# @param model The model object to be trained
# @param optimizer The optimizing equation to use to train the model
# @param input_notes The training input data
# @param truth The actual output for the corresponding input
# @param loss_module Equation for calculating the difference between generated and actual output
# @param num_epochs Number of cycles to train the model
def train_model_loop(model, optimizer, dataloader, loss_module):
    # Set model to train mode
    model.train()

    # Training loop   
    for batch, (source_input_note, ground_truth) in enumerate(dataloader):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        source_input_note = source_input_note.to(device)
        ground_truth = ground_truth.to(device)

        ## Step 2: Run the model on the input data
        preds = model(source_input_note)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, ground_truth)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

        ## Step 6: For every 50th batch, print out the current loss as well # of samples trained
        if batch % 50 == 0:
                    loss, current = loss.item(), batch * 100 + len(source_input_note)
                    print(f"loss: {loss:>7f}  [{current:>5d}/{len(dataloader.dataset):>5d}]")

In [ ]:
# trains the model using the dataloader
def train_model(model, optimizer, training_loader, loss):
    epochs = 50
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_model_loop(model, optimizer, training_loader, loss)
    torch.save(model.state_dict(), 'mlp_model.pt')

In [ ]:
# Get processed training data
training_source_inputs, training_ground_truths = processed_training_datasets()

# Initializing MLPMusicDataset objects
training_MLPMusicDataset = MLPMusicDataset(source_input_data=torch.Tensor(training_source_inputs), ground_truth_data=torch.Tensor(training_source_inputs))
testing_MLPMusicDataset = MLPMusicDataset(source_input_data=torch.Tensor(testing_source_inputs), ground_truth_data=torch.Tensor(training_ground_truths))

# Load data to create batches
training_source_inputs = DataLoader(training_MLPMusicDataset, batch_size=100, shuffle=True)
testing_source_inputs = DataLoader(testing_MLPMusicDataset, batch_size=100, shuffle=False)

In [ ]:
# initialize the MLP
model = MLP(3, 4, 2)

# transfer model to GPU
model.to(device)

In [ ]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [ ]:
# this creates a dictionary of outputs for all pieces in the testing dataset

# @param:
    # model_type: which model to use between "pitch" or "no pitch"
# @returns a dictionary mapping with key of file path and value of csv values
def generate_all_outputs(model_type):
    mlp_model = MLP(3, 4, 2)
    mlp_model.to(device)

    testing_results = {}
    test_paths = []

    file = open(HOME_PATH + '/violin-renderer/src/dataset-paths/testing-input.txt','r')
    lines = file.readlines()
    for line in lines:
        test_paths.append(line.strip())
    
    # generating an output for each piece in the testing input dataset
    for i in range(len(testing_X)):
        new_test_input = np.array(testing_X[i])
        new_test_input[:, 1] = set_duration(new_test_input[:, 0], new_test_input[:, 1])
        new_test_input[:, 0] = scale_data(new_test_input[:, 0])
        new_test_input[:, 1] = scale_data(new_test_input[:, 1])

        if model_type == "pitch":
            new_test_input[:, 2] = scale_pitch(new_test_input[:, 2])
            mlp_model.load_state_dict(torch.load(HOME_PATH + '/violin-renderer/src/models/states/mlp_pitch.pt'))
        elif model_type == "no pitch":
            new_test_input[:, 2] = scale_pitch(new_test_input[:, 2]) * 0
            mlp_model.load_state_dict(torch.load(HOME_PATH + '/violin-renderer/src/models/states/mlp_no_pitch.pt'))
        else:
            print("Please choose the correct model")
            return


        
        new_test_input = torch.Tensor(new_test_input)
        new_test_input = new_test_input.to(device)
        
        y_test = mlp_model(new_test_input)
        y_test = y_test.tolist()
        for j in range(len(y_test)):
            y_test[j].append(testing_X[i][j][2])

        testing_results[test_paths[i]] = y_test

    return testing_results

In [ ]:
# calculates error between generated output and testing truth

# @param:
    # model_type: which model to use between "pitch" or "no pitch"
# @return: MSE values for each song in the testing dataset
def MSE_error(model_type):
    testing_results = generate_all_outputs(model_type)
    loss_values = []
    for output_path, truth in zip(testing_results, testing_y):
        output = testing_results[output_path]

        # print(output_path)

        # pitch was only added so the result is able to be synthesized, we can remove it here
        for i in range(len(output)):
            output[i].pop()
        output = torch.Tensor(output)
        truth = torch.Tensor(truth)
        loss_value = loss(output, truth) / len(output)
        loss_values.append(loss_value)

    return loss_values